**PHASE 2: FEATURE ENGINEERING - Dimensionality Reduction (PCA)**

Purpose: Reduce 10K features → 100 dimensions while preserving variance  
Input: tfidf_matrix.pkl (2.4M × 10K)  
Output: pca_reduced_100d.pkl (2.4M × 100), pca_model.pkl  
Algorithm: Incremental PCA (designed for large datasets)  
Target: Preserve 70-80% of variance in 100 components  
ML Involved: YES - PCA is unsupervised dimensionality reduction  
Runtime: ~15-20 minutes  
Why: Makes clustering feasible, reduces noise, avoids curse of dimensionality

In [1]:
# imports

import pandas as pd
import numpy as np
import os
import sys
from sklearn.decomposition import IncrementalPCA
import joblib
import matplotlib.pyplot as plt

# add project root to path for config

sys.path.append('..')
from config import PCA_N_COMPONENTS, PCA_BATCH_SIZE, RANDOM_STATE

print("✓ All imports loaded")
print(f"PCA Config: n_components={PCA_N_COMPONENTS}, batch_size={PCA_BATCH_SIZE}")

✓ All imports loaded
PCA Config: n_components=100, batch_size=10000


In [2]:
# load TF-IDF matrix and metadata

print("Loading TF-IDF matrix...")
tfidf_matrix = joblib.load('data/processed/tfidf_matrix.pkl')
print(f"✓ Loaded TF-IDF matrix")
print(f"  Shape: {tfidf_matrix.shape}")
print(f"  Papers: {tfidf_matrix.shape[0]:,}")
print(f"  Features: {tfidf_matrix.shape[1]:,}")

# load paper IDs to maintain alignment

df_ids = pd.read_pickle('data/processed/tfidf_paper_ids.pkl')
print(f"\n✓ Loaded paper IDs: {len(df_ids):,}")

Loading TF-IDF matrix...
✓ Loaded TF-IDF matrix
  Shape: (2384622, 3000)
  Papers: 2,384,622
  Features: 3,000

✓ Loaded paper IDs: 2,384,622


In [3]:
# initialize pca
# using Incremental PCA because we have a large datasets
# processes data in batches instead of loading all into memory

pca = IncrementalPCA(
    n_components=PCA_N_COMPONENTS,
    batch_size=PCA_BATCH_SIZE
)

print("✓ Incremental PCA initialized")
print(f"  Target components: {PCA_N_COMPONENTS}")
print(f"  Batch size: {PCA_BATCH_SIZE:,} papers per batch")
print(f"  Total batches: {int(np.ceil(tfidf_matrix.shape[0] / PCA_BATCH_SIZE)):,}")

✓ Incremental PCA initialized
  Target components: 100
  Batch size: 10,000 papers per batch
  Total batches: 239


In [4]:
# fit PCA on TF-IDF matrix in batches
# this step could take ~ 3-4 hours for 2.4M papers

print("\nFitting Incremental PCA...")
print("This will take 3-4 hours...\n")

# convert sparse matrix to dense in batches

n_samples = tfidf_matrix.shape[0]
n_batches = int(np.ceil(n_samples / PCA_BATCH_SIZE))

for i in range(n_batches):
    start_idx = i * PCA_BATCH_SIZE
    end_idx = min(start_idx + PCA_BATCH_SIZE, n_samples)
    
    # get batch and convert to dense
    batch = tfidf_matrix[start_idx:end_idx].toarray()
    
    # partial fit
    pca.partial_fit(batch)
    
    if (i + 1) % 10 == 0 or (i + 1) == n_batches:
        print(f"  Processed batch {i+1}/{n_batches} ({end_idx:,}/{n_samples:,} papers)")

print("\n✓ PCA fitting complete!")


Fitting Incremental PCA...
This will take 3-4 hours...

  Processed batch 10/239 (100,000/2,384,622 papers)
  Processed batch 20/239 (200,000/2,384,622 papers)
  Processed batch 30/239 (300,000/2,384,622 papers)
  Processed batch 40/239 (400,000/2,384,622 papers)
  Processed batch 50/239 (500,000/2,384,622 papers)
  Processed batch 60/239 (600,000/2,384,622 papers)
  Processed batch 70/239 (700,000/2,384,622 papers)
  Processed batch 80/239 (800,000/2,384,622 papers)
  Processed batch 90/239 (900,000/2,384,622 papers)
  Processed batch 100/239 (1,000,000/2,384,622 papers)
  Processed batch 110/239 (1,100,000/2,384,622 papers)


KeyboardInterrupt: 

In [6]:
# transform TF-IDF matrix to PCA space

print("\nTransforming data to reduced space...")

# transform in batches (same as fitting)

X_reduced_list = []

for i in range(n_batches):
    start_idx = i * PCA_BATCH_SIZE
    end_idx = min(start_idx + PCA_BATCH_SIZE, n_samples)
    
    batch = tfidf_matrix[start_idx:end_idx].toarray()
    batch_reduced = pca.transform(batch)
    X_reduced_list.append(batch_reduced)
    
    if (i + 1) % 10 == 0 or (i + 1) == n_batches:
        print(f"  Transformed batch {i+1}/{n_batches}")

# combine all batches

X_reduced = np.vstack(X_reduced_list)

print("\n✓ Transformation complete!")
print(f"\nReduced matrix shape: {X_reduced.shape}")
print(f"  Papers: {X_reduced.shape[0]:,}")
print(f"  Components: {X_reduced.shape[1]}")
print(f"  Memory: {X_reduced.nbytes / 1024**3:.2f} GB")


Transforming data to reduced space...
  Transformed batch 10/239
  Transformed batch 20/239
  Transformed batch 30/239
  Transformed batch 40/239
  Transformed batch 50/239
  Transformed batch 60/239
  Transformed batch 70/239
  Transformed batch 80/239
  Transformed batch 90/239
  Transformed batch 100/239
  Transformed batch 110/239
  Transformed batch 120/239
  Transformed batch 130/239
  Transformed batch 140/239
  Transformed batch 150/239
  Transformed batch 160/239
  Transformed batch 170/239
  Transformed batch 180/239
  Transformed batch 190/239
  Transformed batch 200/239
  Transformed batch 210/239
  Transformed batch 220/239
  Transformed batch 230/239
  Transformed batch 239/239

✓ Transformation complete!

Reduced matrix shape: (2384622, 100)
  Papers: 2,384,622
  Components: 100
  Memory: 1.78 GB


In [7]:
# analyze how much variance we preserved

variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

print(f"Variance Analysis:")
print(f"  Total variance explained: {cumulative_variance[-1]*100:.2f}%")
print(f"  First 10 components: {cumulative_variance[9]*100:.2f}%")
print(f"  First 25 components: {cumulative_variance[24]*100:.2f}%")
print(f"  First 50 components: {cumulative_variance[49]*100:.2f}%")
print(f"\nIndividual component variance:")
print(f"  PC1: {variance_explained[0]*100:.2f}%")
print(f"  PC2: {variance_explained[1]*100:.2f}%")
print(f"  PC3: {variance_explained[2]*100:.2f}%")

Variance Analysis:
  Total variance explained: 10.96%
  First 10 components: 2.89%
  First 25 components: 5.04%
  First 50 components: 7.51%

Individual component variance:
  PC1: 0.59%
  PC2: 0.42%
  PC3: 0.35%


In [ ]:
# plot cumulative variance explained
# visually see if target of 100 components is justified

plt.figure(figsize=(10, 6))
plt.plot(range(1, PCA_N_COMPONENTS + 1), cumulative_variance * 100, 'b-', linewidth=2)
plt.axhline(y=70, color='r', linestyle='--', label='70% variance')
plt.axhline(y=80, color='g', linestyle='--', label='80% variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Variance Explained (%)')
plt.title('PCA Variance Explained')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()

# save plot

os.makedirs('results/figures', exist_ok=True)
plt.savefig('results/figures/pca_variance_explained.png', dpi=150, bbox_inches='tight')
print("\n✓ Variance plot saved to: results/figures/pca_variance_explained.png")
plt.show()

In [ ]:
# save variance explained summary as readable text

print("\nSaving variance analysis summary...")

summary_path = 'results/pca_variance_summary.txt'

with open(summary_path, 'w') as f:
    f.write("PCA DIMENSIONALITY REDUCTION - VARIANCE ANALYSIS\n")
    f.write(f"Analysis date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}\n")
    f.write(f"Input dimensions: {tfidf_matrix.shape[1]:,} features\n")
    f.write(f"Output dimensions: {PCA_N_COMPONENTS} components\n")
    f.write(f"Reduction ratio: {tfidf_matrix.shape[1]/PCA_N_COMPONENTS:.1f}x\n\n")
    
    f.write("VARIANCE EXPLAINED\n")
    f.write(f"Total variance preserved: {cumulative_variance[-1]*100:.2f}%\n")
    f.write(f"Variance lost: {(1-cumulative_variance[-1])*100:.2f}%\n\n")
    
    f.write("Cumulative variance by component count:\n")
    milestones = [1, 5, 10, 25, 50, 75, 100]
    for n in milestones:
        if n <= len(cumulative_variance):
            f.write(f"  First {n:3d} components: {cumulative_variance[n-1]*100:5.2f}%\n")
    
    f.write(f"\n")
    f.write("TOP COMPONENTS (Individual Variance)\n")
    f.write(f"{'Component':<12} {'Variance %':<12} {'Cumulative %'}\n")
    
    for i in range(min(20, len(variance_explained))):
        f.write(f"PC{i+1:<10} {variance_explained[i]*100:>10.2f}% {cumulative_variance[i]*100:>14.2f}%\n")
    
    f.write(f"\n")
    f.write("INTERPRETATION NOTES\n")
    f.write(f"• First component captures {variance_explained[0]*100:.1f}% of variance\n")
    f.write(f"• First 10 components capture {cumulative_variance[9]*100:.1f}% of variance\n")
    f.write(f"• {PCA_N_COMPONENTS} components preserve {cumulative_variance[-1]*100:.1f}% of information\n")
    f.write(f"• Dimensionality reduced from {tfidf_matrix.shape[1]:,} to {PCA_N_COMPONENTS}\n")

print(f"✓ Saved variance summary to: {summary_path}")

In [ ]:
# save outputs

# save top feature loadings per component (for later interpretation)
print("\nSaving component feature loadings...")

# get top features for first 10 components
component_features = {}
feature_names_array = np.array(vectorizer.get_feature_names_out())

for i in range(min(10, PCA_N_COMPONENTS)):
    # get loadings for this component
    component_loading = pca.components_[i]
    
    # top 20 positive loadings
    top_positive_idx = component_loading.argsort()[-20:][::-1]
    top_positive = [(feature_names_array[idx], component_loading[idx]) 
                    for idx in top_positive_idx]
    
    # top 20 negative loadings  
    top_negative_idx = component_loading.argsort()[:20]
    top_negative = [(feature_names_array[idx], component_loading[idx]) 
                    for idx in top_negative_idx]
    
    component_features[f'PC{i+1}'] = {
        'positive': top_positive,
        'negative': top_negative
    }

component_features_path = 'data/processed/pca_component_features.pkl'
joblib.dump(component_features, component_features_path)
print(f"✓ Saved component features to: {component_features_path}")

# save reduced matrix

reduced_path = 'data/processed/pca_reduced_100d.pkl'
joblib.dump(X_reduced, reduced_path)
print(f"✓ Saved reduced matrix to: {reduced_path}")

# save PCA model (for potential inverse transform or analysis)

pca_model_path = 'data/processed/pca_model.pkl'
joblib.dump(pca, pca_model_path)
print(f"✓ Saved PCA model to: {pca_model_path}")

# save variance info

variance_info = {
    'explained_variance_ratio': variance_explained,
    'cumulative_variance': cumulative_variance,
    'total_variance_explained': cumulative_variance[-1]
}
variance_path = 'data/processed/pca_variance_info.pkl'
joblib.dump(variance_info, variance_path)
print(f"✓ Saved variance info to: {variance_path}")

In [ ]:
# verify all files created successfully

files_to_check = [
    'data/processed/pca_reduced_100d.pkl',
    'data/processed/pca_model.pkl',
    'data/processed/pca_variance_info.pkl',
    'data/processed/pca_component_features.pkl',      
    'results/pca_variance_summary.txt',               
    'results/figures/pca_variance_explained.png'
]

all_good = True
for file_path in files_to_check:
    if os.path.exists(file_path):
        size_mb = os.path.getsize(file_path) / 1024**2
        print(f"✓ {file_path}")
        print(f"  Size: {size_mb:.1f} MB")
    else:
        print(f"x Missing: {file_path}")
        all_good = False

if all_good:
    print("\n✓✓✓ Success! ✓✓✓ - All files created!")
    
    # quick reload test
    X_check = joblib.load('data/processed/pca_reduced_100d.pkl')
    pca_check = joblib.load('data/processed/pca_model.pkl')
    
    print(f"\nReload verification:")
    print(f"  Matrix shape: {X_check.shape}")
    print(f"  PCA components: {pca_check.n_components}")
    print(f"  Variance preserved: {cumulative_variance[-1]*100:.2f}%")
else:
    print("\nx Error - Some files missing!")